In [17]:
import pandas as pd
import numpy
from numpy import array
import spacy
from spacy.vocab import Vocab
import keras as k
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense, Dropout, Masking, Embedding
import pickle
import nltk
#nltk.download('stopwords')

In [2]:
#reading processed data
data = open('cleandata.csv').read()[:100000]
data

'status_message\nVamos tomar chopp hoje😍\n"Todo mundo curte um hot roll. E quem negar, mentiu. 😎 \n\nAbrimos às 18h00!\n🎌 Batista Campos - Rua dos Pariquis, 1712\n🎌 Av. Senador Lemos, 356 🎌 Av. Duque de Caxias, 510 \nDelivery:\n2121-4463"\nMesa cheia é mesa alegre!\nNosso Filé com Molho de Vinho é fabuloso! Ele ainda acompanha um delicioso risoto de funghi. Venha apreciar os sabores do Vegas! #VegasRestaurante #VivaVegas\n"Daqui a pouco tem Malino’s Hein!\n🍔🍺🍟😍"\n"Você sabia que além de hambúrgueres,temos também Coquetel de Fruta😲\nIsso mesmo, então vc já sabe☺☺☺"\n"Que esse é um dos pratos mais famosos e respeitados aqui em Belém, todos já sabem. Mas só dá para descobrir o motivo provando. Venha experimentar o Vatapá do Point e se surpreenda! 😉 👏 👌\n\n👉 Point Aquários\n- Municipalidade, nº 897, esquina da Wandenkolk.\n- (91) 3223-1686 / 3085-7229.\n- Domingo – 8h às 17h. Segunda a sábado – 8h às 22h30."\n"30ª ED. DO T.D.O. (Tributo Duplo Oficial)\nLEGIÃO URBANA E CHARLIE BROWN JR\n\nH

In [3]:
#function for preparing text data into sequences for training 
def data_sequencing(data):   
    # integer encode sequences of words
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([data])
    with open('tokenizer.pkl', 'wb') as f: # Save the tokeniser by pickling it
        pickle.dump(tokenizer, f)

    encoded = tokenizer.texts_to_sequences([data])[0]
    # retrieve vocabulary size
    vocab_size = len(tokenizer.word_index) + 1
    print('Vocabulary Size: %d' % vocab_size)
    
    # create line-based sequences
    sequences = list()
    rev_sequences = list()
    for line in data.split('.'):
        encoded = tokenizer.texts_to_sequences([line])[0]
        rev_encoded = encoded[::-1]
        for i in range(1, len(encoded)):
            sequence = encoded[:i+1]
            rev_sequence = rev_encoded[:i+1]
            sequences.append(sequence)
            rev_sequences.append(rev_sequence)
    print('Total Sequences: %d' % len(sequences))
    
    
    #find max sequence length 
    max_length = max([len(seq) for seq in sequences])
    with open('max_length.pkl', 'wb') as f: # Save max_length by pickling it
        pickle.dump(max_length, f)
    print('Max Sequence Length: %d' % max_length)

    # pad sequences and create the forward sequence
    sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
    # split into input and output elements
    sequences = array(sequences)
    X, y = sequences[:,:-1],sequences[:,-1]
    

    return X,y,max_length,vocab_size

In [4]:
#returning forward and reverse sequences along with max sequence 
#length from the data 

X,y,max_length,vocab_size = data_sequencing(data)

Vocabulary Size: 3590
Total Sequences: 15938
Max Sequence Length: 229


In [7]:
# define forward sequence model
model_0 = Sequential()
model_0.add(Embedding(vocab_size,100, input_length=max_length-1))
model_0.add(SimpleRNN(100))
model_0.add(Dense(vocab_size, activation='softmax'))
print(model_0.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 228, 100)          359000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               20100     
_________________________________________________________________
dense (Dense)                (None, 3590)              362590    
Total params: 741,690
Trainable params: 741,690
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
# compile forward sequence network
model_0.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_0.fit(X, y,batch_size=100, epochs=200, verbose=2)
# save the model to file
model_0.save('model_0.h5')

Epoch 1/200
160/160 - 11s - loss: 7.2844 - accuracy: 0.0331
Epoch 2/200
160/160 - 10s - loss: 6.7872 - accuracy: 0.0378
Epoch 3/200
160/160 - 10s - loss: 6.5120 - accuracy: 0.0435
Epoch 4/200
160/160 - 11s - loss: 6.1648 - accuracy: 0.0654
Epoch 5/200
160/160 - 11s - loss: 5.7789 - accuracy: 0.0957
Epoch 6/200
160/160 - 10s - loss: 5.3771 - accuracy: 0.1390
Epoch 7/200
160/160 - 10s - loss: 4.9762 - accuracy: 0.1865
Epoch 8/200
160/160 - 11s - loss: 4.5858 - accuracy: 0.2269
Epoch 9/200
160/160 - 11s - loss: 4.2159 - accuracy: 0.2684
Epoch 10/200
160/160 - 10s - loss: 3.8662 - accuracy: 0.3139
Epoch 11/200
160/160 - 10s - loss: 3.5916 - accuracy: 0.3548
Epoch 12/200
160/160 - 10s - loss: 3.3356 - accuracy: 0.3925
Epoch 13/200
160/160 - 11s - loss: 2.9791 - accuracy: 0.4529
Epoch 14/200
160/160 - 10s - loss: 2.7288 - accuracy: 0.4977
Epoch 15/200
160/160 - 10s - loss: 2.4798 - accuracy: 0.5421
Epoch 16/200
160/160 - 11s - loss: 2.2290 - accuracy: 0.5881
Epoch 17/200
160/160 - 10s - loss

Epoch 135/200
160/160 - 10s - loss: 0.0602 - accuracy: 0.9740
Epoch 136/200
160/160 - 10s - loss: 0.0628 - accuracy: 0.9737
Epoch 137/200
160/160 - 10s - loss: 0.1235 - accuracy: 0.9593
Epoch 138/200
160/160 - 10s - loss: 0.1375 - accuracy: 0.9564
Epoch 139/200
160/160 - 10s - loss: 0.0829 - accuracy: 0.9702
Epoch 140/200
160/160 - 10s - loss: 0.0673 - accuracy: 0.9726
Epoch 141/200
160/160 - 10s - loss: 0.0628 - accuracy: 0.9738
Epoch 142/200
160/160 - 10s - loss: 0.0614 - accuracy: 0.9735
Epoch 143/200
160/160 - 10s - loss: 0.0606 - accuracy: 0.9737
Epoch 144/200
160/160 - 10s - loss: 0.0603 - accuracy: 0.9750
Epoch 145/200
160/160 - 10s - loss: 0.0604 - accuracy: 0.9742
Epoch 146/200
160/160 - 10s - loss: 0.0601 - accuracy: 0.9747
Epoch 147/200
160/160 - 10s - loss: 0.0598 - accuracy: 0.9736
Epoch 148/200
160/160 - 10s - loss: 0.0591 - accuracy: 0.9745
Epoch 149/200
160/160 - 10s - loss: 0.0592 - accuracy: 0.9737
Epoch 150/200
160/160 - 10s - loss: 0.0607 - accuracy: 0.9740
Epoch 15

In [15]:
# generate a sequence using a language model
def generate_seq(model, tokenizer, max_length, seed_text):
    if seed_text == "":
        return ""
    else:
        in_text = seed_text
        n_words = 1
        n_preds = 5 #number of words to predict for the seed text
        pred_words = ""
        # generate a fixed number of words
        for _ in range(n_words):
            # encode the text as integer
            encoded = tokenizer.texts_to_sequences([in_text])[0]
            # pre-pad sequences to a fixed length
            encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
            # predict probabilities for each word
            proba = model.predict(encoded, verbose=0).flatten()
            #take the n_preds highest probability classes 
            yhat = numpy.argsort(-proba)[:n_preds] 
            # map predicted words index to word
            out_word = ''

            for _ in range(n_preds):
                for word, index in tokenizer.word_index.items():
                    if index == yhat[_] and word not in stopwords:
                        out_word = word
                        pred_words += ' ' + out_word
                        #print(out_word)
                        break


        return pred_words

In [18]:
# load the model
model = load_model('model_0.h5')

#load tokeniser and max_length
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
    
with open('max_length.pkl', 'rb') as f:
    max_length = pickle.load(f)
    
#loading stopwords to improve relevant word predictions    
#stopwords= open('stopwords').read().split()
stopwords = nltk.corpus.stopwords.words('portuguese')

#load spacy Model
nlp = spacy.load('pt_core_news_md')

/home/paulamyrian/anaconda3/envs/py36/lib/python3.6/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'pt_core_news_md' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.3.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [19]:
#Find and set embeddings for OOV words
def set_embedding_for_oov(doc):
    #checking for oov words and adding embedding
    for token in doc:
        if token.is_oov == True:
            before_text = doc[:token.i].text

            pred_before = generate_seq(model, tokenizer, max_length-1, before_text).split()
            
            embedding = numpy.zeros((300,))

            i=len(before_text)
            print('Words predicted from forward sequence model:')
            for word in pred_before:
                print(word)
                embedding += i*nlp.vocab.get_vector(word)
                i= i*.5

            nlp.vocab.set_vector(token.text, embedding)
            print(token.text,nlp.vocab.get_vector(token.text)) 

In [20]:
#function to find most similar words
def most_similar(word):
    by_similarity = sorted(word.vocab, key=lambda w: word.similarity(w), reverse=True)
    return [w.orth_ for w in by_similarity[:10]]

# Teste 1

In [21]:
doc1 = nlp('Ana Maria contratou um sanfoneiro paidégua para tocar no aniversário do marido.')
set_embedding_for_oov(doc1)

Words predicted from forward sequence model:
encanto
bolo
pouco
informações
reservas
paidégua [ 1.71009674e+01 -2.04430878e+02 -5.19778900e+01  8.94838333e+01
 -2.99054241e+01 -5.21336479e+01 -1.59186676e+02 -4.92739067e+01
 -7.73614979e+00  1.94981747e+01  2.80358791e+01 -7.16603088e+01
  1.07061287e+02 -7.23020096e+01  4.83176117e+01  5.77188568e+01
  7.08974686e+01 -2.67413464e+01 -6.24737625e+01  1.75639725e+02
 -6.64867477e+01  3.58077202e+01 -1.33833590e+01  1.18247738e+01
  3.60360603e+01 -3.14198151e+01 -1.26321960e+02  1.68701950e+02
  8.46074772e+00 -4.60710526e+01 -1.17793903e+01  1.26182640e+02
  8.72013016e+01 -6.15670662e+01  2.69948215e+01  3.16251602e+01
 -5.45476379e+01  1.27288651e+02  2.16183014e+01  1.45659523e+01
  6.86383533e+00  2.79880857e+00  4.94618454e+01  6.13511696e+01
  5.39077682e+01  1.28182358e+02  9.03514633e+01 -4.87206001e+01
  1.03050552e+02 -5.62147217e+01  4.05613289e+01  1.37586899e+02
 -8.33372650e+01  3.15793724e+01 -7.59407959e+01  4.11688843e

In [22]:
doc2 = nlp('Essa festa tá paidégua.')
set_embedding_for_oov(doc2)

In [23]:
doc3 = nlp('Aquele suco de acerola é paidégua.')
set_embedding_for_oov(doc3)

In [24]:
doc4 = nlp('Ana Maria contratou um sanfoneiro paidégua para tocar no aniversário do marido.')
doc5 = nlp('Camões foi um homem valoroso.')

doc6 = nlp('Essa festa tá paidégua.')
doc7 = nlp('É muito legal brincar no computador.')

doc8 = nlp('Aquele suco de acerola é paidégua.')
doc9 = nlp('Este lanche está gostoso.')

In [26]:
print(doc4, "<->", doc5, "<->",  doc4.similarity(doc5))
print(doc6, "<->", doc7, "<->",  doc6.similarity(doc7))
print(doc8, "<->", doc9, "<->",  doc8.similarity(doc9))

Ana Maria contratou um sanfoneiro paidégua para tocar no aniversário do marido. <-> Camões foi um homem valoroso. <-> 0.36093655614157194
Essa festa tá paidégua. <-> É muito legal brincar no computador. <-> 0.3023453572674754
Aquele suco de acerola é paidégua. <-> Este lanche está gostoso. <-> 0.3857092028351202


In [27]:
nlp('paidégua').similarity(nlp('gostoso'))

0.5348792870713445

In [28]:
most_similar(nlp('paidégua'))

/home/paulamyrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  This is separate from the ipykernel package so we can avoid doing imports until


['paidégua',
 'muito',
 'gostoso',
 'acerola',
 'brincar',
 'festa',
 'lanche',
 '^__^',
 ':-/',
 'valoroso']